In [1]:
search_for = 139
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0900270938873291
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 376006121
type: [1, 1, 1, 1, 139] 139
cases of this type: 373301041
100000 12.357987457732335
200000 19.206382902348192
300000 22.772984294645497
400000 19.69351993000599
500000 20.576347674854283
600000 19.298817997410588
700000 23.24124710969541
800000 21.805434037210517
900000 19.966147418065642
1000000 17.33802345918989
1100000 21.49607294394428
1200000 19.727241341413347
1300000 20.77342396325208
1400000 3.793245696065005
1500000 22.241639437463157
1600000 17.405303338986588
1700000 18.86518373503913
1800000 20.792295554768334
1900000 16.466599139253823
2000000 21.637160883531674
2100000 22.690434818094758
2200000 25.576032731421883
2300000 27.720734390510373
2400000 35.48848287387455
2500000 32.770599164721474
2600000 35.773836084625714
2700000 18.065988741707528
2800000 24.193572114462825
2900000 40.32338750874758
3000000 46

29700000 36.41430934542131
29800000 12.608904744455852
29900000 32.34419528349973
30000000 36.09096902606962
30100000 34.7803992753226
30200000 36.6278354795974
30300000 33.128020526691344
30400000 35.99575661256848
30500000 33.54389070665124
30600000 35.19278631301341
30700000 32.27084494692592
30800000 32.50523631337345
30900000 16.21305957991589
31000000 26.179106387479298
31100000 30.64695534469419
31200000 34.83011586160876
31300000 31.184351954798206
31400000 35.11786051086409
31500000 31.245160273565716
31600000 34.116690946540174
31700000 32.04215512651505
31800000 34.282567146194154
31900000 32.953603185494885
32000000 24.02353323793036
32100000 23.21725922497014
32200000 34.519640338481786
32300000 35.05058868713208
32400000 35.99475968026383
32500000 13.841267776589476
32600000 31.24868544699402
32700000 37.40207973727039
32800000 35.98826465954638
32900000 34.483799236529606
33000000 32.867958858801586
33100000 34.258870647430456
33200000 33.386758344735625
33300000 33.3595

59700000 32.51929782845475
59800000 32.23048188524707
59900000 31.317654397270232
60000000 32.12304295954956
60100000 29.519024331658557
60200000 31.643334550653
60300000 28.30456034436023
60400000 26.572405082107316
60500000 12.173430879648139
60600000 30.30545917203079
60700000 28.96258953953689
60800000 30.999660389176608
60900000 28.94133064274734
61000000 32.30998880828458
61100000 29.890052735814326
61200000 31.999476660496267
61300000 31.67857124074944
61400000 12.85097796560889
61500000 33.87481605689021
61600000 31.319399484555333
61700000 33.519693953809764
61800000 31.715260120331916
61900000 35.87944051416714
62000000 34.84259065564414
62100000 33.849825847925764
62200000 29.25079475418205
62300000 16.20633950239975
62400000 33.61228494533233
62500000 33.92921382632901
62600000 31.202151281316294
62700000 35.634552760332966
62800000 29.84582360884446
62900000 31.649770053401184
63000000 31.00215864735235
63100000 20.924347910910765
63200000 16.50573864164366
63300000 30.384

89500000 26.55154200040835
89600000 26.735909684768338
89700000 26.91146583800532
89800000 25.90488338863512
89900000 26.471723859240118
90000000 8.807420412210519
90100000 24.616936465133765
90200000 26.121726459787702
90300000 25.861795590959126
90400000 25.874830307960593
90500000 25.835292172266385
90600000 26.074204996279708
90700000 10.658005561352264
90800000 26.947506955429198
90900000 28.133566077814308
91000000 27.91075915882424
91100000 28.5934806163599
91200000 28.271076427230923
91300000 30.22829231271266
91400000 30.399425411731748
91500000 29.13326217529895
91600000 29.078303994634823
91700000 29.368398998123418
91800000 28.92610429266386
91900000 28.09869885560145
92000000 13.15769482610506
92100000 21.75794292802356
92200000 26.78037465445635
92300000 25.856406940784346
92400000 25.09735428399251
92500000 25.462768994661445
92600000 26.371151014874087
92700000 4.428434398144925
92800000 27.477160769907037
92900000 26.012226780392275
93000000 25.020832058555985
93100000

118600000 24.826782271704673
118700000 25.002382084879684
118800000 24.006224629049605
118900000 23.373124535949174
119000000 19.900398382859905
119100000 10.747929961526076
119200000 22.746565150879515
119300000 22.11481473669095
119400000 22.760489074428193
119500000 16.052752047119938
119600000 13.282947973139208
119700000 22.434447529935436
119800000 22.51929706663593
119900000 22.555492308721476
120000000 14.667617033706703
120100000 17.436996871137595
120200000 23.543109449335518
120300000 22.855690832505786
120400000 24.160144974984597
120500000 23.567024253410032
120600000 24.83075179993055
120700000 25.612464799564545
120800000 25.392088867245466
120900000 25.4638901302765
121000000 26.226267967360467
121100000 25.86709471197924
121200000 26.10045869993199
121300000 25.188032848392986
121400000 26.205735286477086
121500000 24.17088230392051
121600000 23.719119821266435
121700000 21.72442090505588
121800000 9.237625070042812
121900000 23.37513590984224
122000000 24.009264816017

...

233200000 13.508863116185424
233300000 14.823975786368734
233400000 14.71161754781979
233500000 15.799864908819922
233600000 14.793566233131662
233700000 15.199227301815634
233800000 14.511966693041986
233900000 15.484742481353615
234000000 14.973621406153509
234100000 15.114614655637672
234200000 13.670405236117409
234300000 14.438232131897642
234400000 13.017287728213953
234500000 14.094055832497475
234600000 13.342953450109416
234700000 4.792879866090334
234800000 14.221278602416433
234900000 13.443628884944573
235000000 7.299886836655882
235100000 9.650524342059228
235200000 14.746394919753723
235300000 12.910958217916564
235400000 4.5988946945993145
235500000 13.455869508302516
235600000 14.056431564349168
235700000 13.355495164659406
235800000 14.786185624818076
235900000 13.797790642333485
236000000 15.374300614385772
236100000 14.928928365893263
236200000 15.968316998768868
236300000 14.33407159406921
236400000 15.192914062763409
236500000 13.681278846987437
236600000 15.888977

261800000 10.932212102894676
261900000 1.79542675693476
262000000 11.013684340368174
262100000 11.371079563612607
262200000 10.550472358137545
262300000 11.301021322600855
262400000 6.129564385240162
262500000 8.310452638837301
262600000 10.832475806364076
262700000 11.957450428980358
262800000 11.350104706482407
262900000 12.754517982851734
263000000 11.252117229710429
263100000 12.251683535678644
263200000 11.740003216770393
263300000 11.684755054650461
263400000 11.592371728665002
263500000 11.194250457771586
263600000 11.237504301679689
263700000 10.996346628814418
263800000 11.113939073358518
263900000 10.1740332833633
264000000 4.830810991698653
264100000 8.983029645852081
264200000 10.718981156314017
264300000 9.726302503013414
264400000 10.774022973627298
264500000 9.094607798694796
264600000 3.393998566370514
264700000 10.111992460346817
264800000 11.199591785070846
264900000 9.565404446010572
265000000 10.703990269507472
265100000 6.282423167519989
265200000 8.289406161045923

290900000 8.518862822896036
291000000 7.711298883591246
291100000 8.21568300361397
291200000 7.78125865103215
291300000 8.112958203784133
291400000 4.256873765847511
291500000 6.065248467385447
291600000 7.802554581748823
291700000 8.031034159328868
291800000 7.873282371728164
291900000 8.074998836116599
292000000 8.56809136484386
292100000 8.451095157505975
292200000 2.528283222094296
292300000 8.64825719343245
292400000 9.325052016794784
292500000 8.604226690492457
292600000 9.476892948167334
292700000 8.721023916075547
292800000 9.319001390874392
292900000 8.44557512559125
293000000 9.600933496590255
293100000 8.009727628206347
293200000 9.243525652086733
293300000 5.549939575776285
293400000 5.749328139358218
293500000 7.869197299855635
293600000 8.57360340917534
293700000 7.477467724030171
293800000 8.21797363069998
293900000 7.7810378487723275
294000000 7.994381089366811
294100000 3.0777077030568925
294200000 6.841953403084225
294300000 7.707058274583574
294400000 8.0751871948445

320200000 5.9861083328842835
320300000 6.008080644616341
320400000 5.7430136125346705
320500000 5.548061232635572
320600000 5.853466119254142
320700000 5.2998334295990865
320800000 5.485357277805862
320900000 4.645706365659841
321000000 1.9522185765513544
321100000 5.18164221190859
321200000 5.649354218379467
321300000 5.148517977144711
321400000 6.279498227467973
321500000 5.457823501270849
321600000 6.178481197718042
321700000 5.526039097935913
321800000 6.384236711273737
321900000 3.5551918191394476
322000000 4.037440409974748
322100000 5.557460808204371
322200000 6.077054101022299
322300000 5.361536813223914
322400000 5.861744401358273
322500000 5.3433951937073605
322600000 5.100422053631902
322700000 2.6455115536103864
322800000 5.48057010137515
322900000 5.420462905563207
323000000 5.214378714552927
323100000 5.208074806566527
323200000 4.980345253013935
323300000 5.410520156518946
323400000 4.616319182348472
323500000 4.972696554271594
323600000 3.6652304164729896
323700000 2.52

349200000 2.796189175922971
349300000 1.75922558323784
349400000 2.1552220584958595
349500000 2.600000671141022
349600000 2.780059957084115
349700000 2.4632418322110374
349800000 2.850478178956094
349900000 2.383011592894382
350000000 2.783397677750981
350100000 2.395979295277017
350200000 2.709476443071182
350300000 2.3712609938691975
350400000 2.5154325548296126
350500000 0.9274773298215521
350600000 2.08569362528581
350700000 2.4131776272420216
350800000 2.6188974119615787
350900000 2.433775136936587
351000000 2.5220834221916535
351100000 2.4125953851586255
351200000 2.5932865228296915
351300000 2.5489501754581965
351400000 2.476118565596405
351500000 2.548647932499224
351600000 2.639783012115227
351700000 0.5707518140015284
351800000 2.5725539754635096
351900000 2.500666100568844
352000000 0.8816588138586898
352100000 2.4419511689530236
352200000 2.3914848929413237
352300000 2.3636108985647404
352400000 2.3167882921363674
352500000 2.4755431124436242
352600000 2.2244879653642933
35